In [1]:
!pip install tensorflow-text

print('\nBibliotecas instaladas com sucesso!')


Bibliotecas instaladas com sucesso!


In [2]:
import pandas as pd
import numpy as np
import tensorflow_text
import tensorflow_hub as hub
import joblib as jb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report

print('Bibliotecas importadas com sucesso!')

Bibliotecas importadas com sucesso!


In [3]:
# Carregando o universal sentence encoder puxando os dados da web
embed = hub.load('https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/multilingual-large/2')

In [4]:
dataset = pd.read_csv('dataset_complaints.csv')

In [5]:
# Fazendo o embedding de todas as frases do dataset
x = []
for i in range(len(dataset['desc_problema'])):
  x.append(embed(dataset['desc_problema'][i]))

In [6]:
# Fazendo embedding da frase do usuario e relacionando-a com as frases do dataset para gerar um array.

frase_usuario = "Meu carro está apresentando problemas e sons estranhos."

usuario_embed = embed(frase_usuario)
resultado_frases = []
numeros = []

for i in x:
  resultado = np.inner(i, usuario_embed)
  resultado_frases.append(resultado.flatten()[0])

# Adicionando os arrays gerados no dataset
dataset['desc_problema'] = resultado_frases
dataset['desc_problema']

,desc_problema
0,0.261119
1,0.229338
2,0.242454
3,0.200468
4,0.269125
...,...
289,0.422076
290,0.244925
291,0.357399
292,0.425391


In [7]:
# Utilizando o label encoder para transformar dados não numericos em numeros serializados.

label_encoder = LabelEncoder()
dataset['modelo_carro'] = label_encoder.fit_transform(dataset['modelo_carro'])
dataset['peca_problema'] = label_encoder.fit_transform(dataset['peca_problema'])
dataset

,desc_problema,ano,modelo_carro,categoria_problema,peca_problema
0,0.261119,2012,3,Disco/Pastilha de Freio,20
1,0.229338,2018,3,Disco/Pastilha de Freio,20
2,0.242454,2015,0,Disco/Pastilha de Freio,20
3,0.200468,2013,1,Disco/Pastilha de Freio,20
4,0.269125,2021,2,Disco/Pastilha de Freio,20
...,...,...,...,...,...
289,0.422076,2018,2,Balanceamento/Geometria,2
290,0.244925,2019,2,Balanceamento/Geometria,2
291,0.357399,2022,1,Balanceamento/Geometria,2
292,0.425391,2023,1,Balanceamento/Geometria,2


In [8]:
# Separando dados de treino / teste

dataset_alt = dataset.drop(columns=['categoria_problema'])
x = dataset_alt
y = dataset['categoria_problema']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [9]:
# Aplicando os dados no modelo RF (Random Forest)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(x_train, y_train)


RandomForestClassifier(random_state=42)

In [10]:
# Exibindo o relatorio de precisão do modelo

y_predicao = model.predict(x_test)

print("\nRelatório de Classificação:\n")
print(classification_report(y_test, y_predicao))


Relatório de Classificação:

                         precision    recall  f1-score   support

            Alinhamento       1.00      1.00      1.00        12
        Ar Condicionado       0.79      1.00      0.88        11
          Arrefecimento       1.00      0.80      0.89         5
Balanceamento/Geometria       1.00      1.00      1.00         7
               Correias       1.00      0.57      0.73         7
Disco/Pastilha de Freio       0.88      1.00      0.93         7
             Embreagens       1.00      1.00      1.00         5
 Filtro/Vela de Ignição       1.00      1.00      1.00         5

               accuracy                           0.93        59
              macro avg       0.96      0.92      0.93        59
           weighted avg       0.95      0.93      0.93        59



In [11]:
# Teste de predição do modelo

for real, predito in zip(y_test, y_predicao):
    print(f"Real: {real}  ----- Predito: {predito}")

Real: Arrefecimento  ----- Predito: Arrefecimento
Real: Disco/Pastilha de Freio  ----- Predito: Disco/Pastilha de Freio
Real: Correias  ----- Predito: Ar Condicionado
Real: Balanceamento/Geometria  ----- Predito: Balanceamento/Geometria
Real: Filtro/Vela de Ignição  ----- Predito: Filtro/Vela de Ignição
Real: Ar Condicionado  ----- Predito: Ar Condicionado
Real: Disco/Pastilha de Freio  ----- Predito: Disco/Pastilha de Freio
Real: Disco/Pastilha de Freio  ----- Predito: Disco/Pastilha de Freio
Real: Disco/Pastilha de Freio  ----- Predito: Disco/Pastilha de Freio
Real: Correias  ----- Predito: Correias
Real: Alinhamento  ----- Predito: Alinhamento
Real: Correias  ----- Predito: Ar Condicionado
Real: Filtro/Vela de Ignição  ----- Predito: Filtro/Vela de Ignição
Real: Embreagens  ----- Predito: Embreagens
Real: Ar Condicionado  ----- Predito: Ar Condicionado
Real: Arrefecimento  ----- Predito: Disco/Pastilha de Freio
Real: Embreagens  ----- Predito: Embreagens
Real: Alinhamento  ----- Pre

In [ ]:
# Preparando o arquivo para exportação

nome_arq = 'predict_complaints.joblib'

with open(nome_arq, 'wb') as file:
  jb.dump(model, file)

print('Modelo salvo!')

Modelo salvo!
